In [1]:
import logging
import string
from zemberek import TurkishSpellChecker, TurkishSentenceNormalizer, TurkishSentenceExtractor, TurkishMorphology, TurkishTokenizer
import pandas as pd
import re

# Preprocessing

In [7]:
logger = logging.getLogger(__name__)

#Read Turkish stopwords
my_file = open("stopwords.txt","r", encoding="UTF-8")
turkish_stopwords = my_file.read()
turkish_stopwords = turkish_stopwords.replace("\n", " ").split(" ")

#Read Excel file
twitter_analiz = pd.read_excel("Grup1_Etiketleme.xlsx")
df = twitter_analiz.copy()
tweets = df["Tweet"]

#Clear all emojis
def remove_emoji(string):
    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)

    return re.sub(emoj, '', string)

#Clear all links
def remove_urls(text):
    result = re.sub(r'http\S+', '', text, flags=re.MULTILINE)

    return result

def remove_hastaghes(text):
    result = re.sub(r'#\S+', '', text, flags=re.MULTILINE)

    return result

#Make lower case
tweets = tweets.apply(lambda x: str(x).lower())
#Delete all punctuation
tweets = tweets.apply(lambda x: str(x).translate(str.maketrans("", "", string.punctuation)))
#Delete all numbers from string
tweets = tweets.str.replace("\d+", "")
#Remove all emojis
tweets = tweets.apply(lambda x: remove_emoji(x))
#Remove all URLs
tweets = tweets.apply(lambda x: remove_urls(x))
#Remove all hastaghes
tweets = tweets.apply(lambda x: remove_hastaghes(x))
#Delete all Turkish stopwords
tweets = tweets.apply(lambda x: x if x not in turkish_stopwords else None)

#Make sentence
str_tweets = ""

for i in range(len(tweets)):
    str_tweets += str(tweets[i]).replace("\n", "") + " "

#Create morphology and normalizer objects
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

analysis_words = morphology.analyze_sentence(str_tweets)
disambiguate_words = morphology.disambiguate(str_tweets, analysis_words)

#Make series
tweets_word_roots = ""

for i in disambiguate_words.best_analysis():
    if(str(i.item.root) != "UNK"):
        tweets_word_roots += str(i.item.root) + " "

#Splitting by words
tweets_word_roots = tweets_word_roots.split(" ")
#Make Pandas Series
tweets_word_roots_series = pd.Series(tweets_word_roots)
#Make Pandas DataFrame
tweets_words_root_df = pd.DataFrame(tweets_word_roots_series, columns=["Root"])

#Saving DataFrame as Excel file
tweets_words_root_df.to_excel("tweets_words_roots.xlsx")

C:\Users\ERENGULBAHAR\AppData\Local\Temp\ipykernel_22144\2950316622.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace("\d+", "")


2023-06-02 15:26:36,604 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 7.066076278686523



# Concat tweets and have been most frequency roots

In [2]:
import pandas as pd
from zemberek import TurkishSentenceNormalizer, TurkishMorphology
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

In [19]:
#Get uniqe roots
uniqeRoots = pd.read_excel("tweets_words_roots.xlsx");
uniqeRoots = uniqeRoots["UniqueRoot"]
uniqeRoots.dropna(inplace=True)
uniqeRoots

0          kargo
1          kadın
2             bu
3            tür
4           foto
          ...   
2975    resmiyet
2976       doğma
2977     gerekli
2978      hamile
2979         düz
Name: UniqueRoot, Length: 2980, dtype: object

In [33]:
#Get tweets
tweets = pd.read_excel("Grup1_Etiketleme.xlsx")
tweets = tweets["Tweet"]
tweets

0                                   @labirsev Kargo kadın
1       @ElfesyaSeher Bu tür fotoların yayınlanıp olağ...
2       beni çocukluğuma götüren kokulardan birisi pla...
3                                         Ana valla sapık
4                   @NoContextSag Sapik Anadolu muslumani
                              ...                        
996     Yaratılışa aykırı olan her düşünce yaşadığımız...
997     Leonarda da Vinci eşcinsel ve veganmış. Çağımı...
998     Ahmet YILDIZ'A Adalet! , 15 Temmuz 2008 tarihi...
999     Ailede birinin eşcinsel doğması gerekliydi ve ...
1000    @jeandpardaillan El hareketi günümüzde kabul e...
Name: Tweet, Length: 1001, dtype: object

In [30]:
#We saving tweets as like cleaning from unnecessary things
logger = logging.getLogger(__name__)

#Read Turkish stopwords
my_file = open("stopwords.txt","r", encoding="UTF-8")
turkish_stopwords = my_file.read()
turkish_stopwords = turkish_stopwords.replace("\n", " ").split(" ")

#Read Excel file
twitter_analiz = pd.read_excel("Grup1_Etiketleme.xlsx")
df = twitter_analiz.copy()
tweets = df["Tweet"]

#Clear all emojis
def remove_emoji(string):
    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)

    return re.sub(emoj, '', string)

#Clear all links
def remove_urls(text):
    result = re.sub(r'http\S+', '', text, flags=re.MULTILINE)

    return result

def remove_hastaghes(text):
    result = re.sub(r'#\S+', '', text, flags=re.MULTILINE)

    return result

#Make lower case
tweets = tweets.apply(lambda x: str(x).lower())
#Delete all punctuation
tweets = tweets.apply(lambda x: str(x).translate(str.maketrans("", "", string.punctuation)))
#Delete all numbers from string
tweets = tweets.str.replace("\d+", "")
#Remove all emojis
tweets = tweets.apply(lambda x: remove_emoji(x))
#Remove all URLs
tweets = tweets.apply(lambda x: remove_urls(x))
#Remove all hastaghes
tweets = tweets.apply(lambda x: remove_hastaghes(x))
#Delete all Turkish stopwords
tweets = tweets.apply(lambda x: x if x not in turkish_stopwords else None)

tweets.to_csv("cleaned_tweets.csv")

C:\Users\ERENGULBAHAR\AppData\Local\Temp\ipykernel_22144\2160291558.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace("\d+", "")


In [79]:
#We create list by roots
columns = list(uniqeRoots)

#We get cleaned tweets
cleaned_tweets = pd.read_csv("cleaned_tweets.csv")
cleaned_tweets = cleaned_tweets["Tweet"]
cleaned_tweets

0                                    labirsev kargo kadın
1       elfesyaseher bu tür fotoların yayınlanıp olağa...
2       beni çocukluğuma götüren kokulardan birisi pla...
3                                         ana valla sapık
4                    nocontextsag sapik anadolu muslumani
                              ...                        
996     yaratılışa aykırı olan her düşünce yaşadığımız...
997     leonarda da vinci eşcinsel ve veganmış çağımız...
998     ahmet yildiza adalet   temmuz  tarihinde babas...
999     ailede birinin eşcinsel doğması gerekliydi ve ...
1000    jeandpardaillan el hareketi günümüzde kabul ed...
Name: Tweet, Length: 1001, dtype: object

In [269]:
#Read Excel file that has all roots
df = pd.read_excel("tweets_words_roots.xlsx")
df = df["Root"]

#We calculate frequency each of words
information_gain = df.value_counts(normalize=True)

#We make dictionary structure for words and frequencies
temp_dictionary = dict(information_gain)
most_words = list(temp_dictionary.keys())
temp_list = list()

for i in range(1010):
    temp_list.append(most_words[i])

best_words = pd.Series(temp_list)

best_words.to_excel("most_roots.xlsx")

In [270]:
#We get have most frequency roots
most_roots = pd.read_excel("most_roots.xlsx")
most_roots = most_roots["Roots"]
most_roots

0             ol
1       eşcinsel
2          sapık
3             bu
4             de
          ...   
996         alev
997        zafer
998        yaşam
999        oktar
1000       ciddi
Name: Roots, Length: 1001, dtype: object

In [66]:
#Create new excel file including unique roots and all tweets
tweets_matrix = pd.DataFrame(columns=[most_roots], index=[tweets])
tweets_matrix.fillna(0, inplace=True)
tweets_matrix

Roots,ol,eşcinsel,sapık,bu,de,bir,ve,et,ben,yap,...,aşağı,dikkat,cuma,yasak,imkan,mezhep,alev,zafer,yaşam,oktar
Tweet,,,,,,,,,,,,,,,,,,,,,
@labirsev Kargo kadın,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"@ElfesyaSeher Bu tür fotoların yayınlanıp olağanlaştırılmasına,cinsel hayata saygısızlığa karşıyım…eminim ki sapık papazların da bir çok fotosu vardır da ben henüz rastlamadım..din kutsaldır;kirletilmemeli …",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
beni çocukluğuma götüren kokulardan birisi playstation 1 kokusu ve yeni macbook prolar aynen öyle kokuyor. sapık gibi koklayıp mutlu oluyorum.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ana valla sapık,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
@NoContextSag Sapik Anadolu muslumani,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Yaratılışa aykırı olan her düşünce yaşadığımız yüzyıla hakim olan sistemin bir ürünüdür. Yaradan insanı kadın ve erkek olarak, hem etçil hem de otçul yaratmıştır. Doğuştan eşcinsel olunmayacağı gibi doğuştan vegan da olunmaz. İnsanoğlu doğal dengeden tamamen koparılmak isteniyor.",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Leonarda da Vinci eşcinsel ve veganmış. Çağımızda yaşasaydı twitterda neler yazardı çok merak ediyorum.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"Ahmet YILDIZ'A Adalet! , 15 Temmuz 2008 tarihinde babası tarafından Üsküdar’da “sırf” eşcinsel olduğu için katledilmiştir! 8 Eylül 2009 tarihinde açılan Ahmet YILDIZ cinayeti davası, Türkiye’de ilk defa eşcinsel namus cinayeti kavramını resmiyete taşıdı. https://t.co/iHx1H30KrE",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
cleaned_tweets

0                                    labirsev kargo kadın
1       elfesyaseher bu tür fotoların yayınlanıp olağa...
2       beni çocukluğuma götüren kokulardan birisi pla...
3                                         ana valla sapık
4                    nocontextsag sapik anadolu muslumani
                              ...                        
996     yaratılışa aykırı olan her düşünce yaşadığımız...
997     leonarda da vinci eşcinsel ve veganmış çağımız...
998     ahmet yildiza adalet   temmuz  tarihinde babas...
999     ailede birinin eşcinsel doğması gerekliydi ve ...
1000    jeandpardaillan el hareketi günümüzde kabul ed...
Name: Tweet, Length: 1001, dtype: object

# Create frequency matrix

In [74]:
#We create frequency matrix
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

counter = 0

for i in range(len(cleaned_tweets)):
    analysis_words = morphology.analyze_sentence(str(cleaned_tweets[i]))
    disambiguate_words = morphology.disambiguate(str(cleaned_tweets[i]), analysis_words)
    tweets_word_roots = ""

    for j in disambiguate_words.best_analysis():
        if (str(j.item.root) != "UNK"):
            tweets_word_roots += str(j.item.root) + " "

    temp_roots_list = tweets_word_roots.split(" ")

    matrix = [0] * (len(most_roots))
    
    for k in range(len(most_roots)):
        value = 0
        
        for l in range(len(temp_roots_list)):
            if(temp_roots_list[l] == most_roots[k]):
                value += 1
                matrix[k] = value

    tweets_matrix.iloc[i:i+1, :] = matrix

    counter += 1

    print(counter)

tweets_matrix.to_csv("tweets_matrix.csv")

2023-06-02 17:03:57,083 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 7.008368968963623

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
24

In [4]:
tweets_matrix = pd.read_excel("tweets_matrix_excel.xlsx")
tweets_matrix_columns = tweets_matrix.columns

In [5]:
X = tweets_matrix.loc[:, tweets_matrix_columns[1:]]
Y = tweets_matrix.loc[:, ["Tweets"]]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

# KNN Cosine Model

In [ ]:
#We do standartization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=1, algorithm="brute", metric="cosine")
knn.fit(X_train, Y_train)

#We predict with our model
Y_pred = knn.predict(X_test)
#We calculate accuracy
accuracy = accuracy_score(Y_test, Y_pred)
#We calculate f1 score
f1_scores = f1_score(Y_test, Y_pred, average="weighted")

print("Accuracy: ", accuracy)
print("F1 Score (Weighted): ", f1_scores)

# TF-IDF

In [271]:
morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)

tweets_list = list()

for i in range(len(cleaned_tweets)):
    analysis_words = morphology.analyze_sentence(str(cleaned_tweets[i]))
    disambiguate_words = morphology.disambiguate(str(cleaned_tweets[i]), analysis_words)
    tweets_word_roots = ""

    for j in disambiguate_words.best_analysis():
        if (str(j.item.root) != "UNK"):
            tweets_word_roots += str(j.item.root) + " "
    
    tweets_list.append(tweets_word_roots)

2023-06-06 04:07:03,082 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 6.691812992095947



In [272]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [273]:
tf_idf_model = TfidfVectorizer()
tf_idf_vector = tf_idf_model.fit_transform(tweets_list)

In [274]:
#We take tf-idf values
tf_idf_array = tf_idf_vector.toarray()

In [275]:
#We take roots
tf_idf_roots = list(tf_idf_model.get_feature_names_out())

In [276]:
temp_df_tf_idf = pd.DataFrame(tf_idf_array, columns=tf_idf_roots)

In [282]:
df_tf_idf = pd.read_excel("temp_cleaned_tweets.xlsx")
df_tf_idf.columns = most_roots

In [286]:
df_tf_idf.fillna(0, inplace=True)

In [288]:
most_words_list = list(most_roots)

In [291]:
#We create new dataframe
for i in range(count_roots):
    df_tf_idf[most_roots[i]] = temp_df_tf_idf[most_roots[i]]

In [331]:
df_tf_idf.insert(0, "Tweets", cleaned_tweets)

In [333]:
df_tf_idf.head()

Roots,Tweets,ol,eşcinsel,sapık,bu,de,bir,ve,et,ben,...,dikkat,cuma,yasak,imkan,mezhep,alev,zafer,yaşam,oktar,ciddi
0,labirsev kargo kadın,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,elfesyaseher bu tür fotoların yayınlanıp olağa...,0.000000,0.0,0.081884,0.092466,0.0,0.106136,0.000000,0.0,0.128971,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,beni çocukluğuma götüren kokulardan birisi pla...,0.071309,0.0,0.069566,0.000000,0.0,0.000000,0.101478,0.0,0.109569,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ana valla sapık,0.000000,0.0,0.231455,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nocontextsag sapik anadolu muslumani,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [334]:
df_tf_idf.to_excel("tf_idf_matrix.xlsx")

In [3]:
#Read TF-IDF tweets matrix
df = pd.read_excel("tf_idf_matrix.xlsx")

In [4]:
df

,Tweets,ol,eşcinsel,sapık,bu,de,bir,ve,et,ben,...,dikkat,cuma,yasak,imkan,mezhep,alev,zafer,yaşam,oktar,ciddi
0,labirsev kargo kadın,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,elfesyaseher bu tür fotoların yayınlanıp olağa...,0.000000,0.000000,0.081884,0.092466,0.000000,0.106136,0.000000,0.000000,0.128971,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,beni çocukluğuma götüren kokulardan birisi pla...,0.071309,0.000000,0.069566,0.000000,0.000000,0.000000,0.101478,0.000000,0.109569,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,ana valla sapık,0.000000,0.000000,0.231455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,nocontextsag sapik anadolu muslumani,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,yaratılışa aykırı olan her düşünce yaşadığımız...,0.280237,0.052573,0.000000,0.000000,0.069194,0.070871,0.079760,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
997,leonarda da vinci eşcinsel ve veganmış çağımız...,0.000000,0.093371,0.000000,0.000000,0.000000,0.000000,0.141656,0.142256,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
998,ahmet yildiza adalet temmuz tarihinde babas...,0.062337,0.116945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
999,ailede birinin eşcinsel doğması gerekliydi ve ...,0.000000,0.117383,0.000000,0.000000,0.000000,0.000000,0.178086,0.000000,0.192285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [177]:
X = df.iloc[:, 1:]
Y = df.loc[:, ["Tweets"]]

In [178]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

# SVM

In [141]:
from sklearn.svm import SVC

In [169]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [170]:
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, Y_train.values.ravel())

SVC(kernel='linear')

In [171]:
Y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
f1_scores = f1_score(Y_test, Y_pred, average="weighted")

In [172]:
print(f"Accuracy Score: {accuracy}\nF1 Score: {f1_scores}")

Accuracy Score: 0.019933554817275746
F1 Score: 0.01882613510520487


# WordToVec-CBOW

In [ ]:
!pip install gensim

In [52]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize

In [87]:
tweets_sentences = ""

for i in df["Tweets"]:
    tweets_sentences += i + " "

In [88]:
data = []
 
for i in sent_tokenize(tweets_sentences):
    temp = []
     
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
 
    data.append(temp)

In [89]:
#Create Word2Vec-CBOW
wordToVec_cbow_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5, sg=0)

2023-06-07 16:44:25,908 - gensim.models.word2vec - INFO
Msg: collecting all words and their counts

2023-06-07 16:44:25,909 - gensim.models.word2vec - INFO
Msg: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types

2023-06-07 16:44:25,913 - gensim.models.word2vec - INFO
Msg: collected 9064 word types from a corpus of 20165 raw words and 1 sentences

2023-06-07 16:44:25,914 - gensim.models.word2vec - INFO
Msg: Creating a fresh vocabulary

2023-06-07 16:44:25,936 - gensim.utils - INFO
Msg: Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 9064 unique words (100.00% of original 9064, drops 0)', 'datetime': '2023-06-07T16:44:25.936901', 'gensim': '4.3.1', 'python': '3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'prepare_vocab'}

2023-06-07 16:44:25,937 - gensim.utils - INFO
Msg: Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 20165 word corpus (100.00% of origi

In [90]:
similarity_words = wordToVec_cbow_model.wv.most_similar("fotoların")

In [91]:
similarity_words

[('çekicek', 0.38024553656578064),
 ('vekilleri', 0.35367411375045776),
 ('pilotları', 0.3501938283443451),
 ('basın', 0.33198702335357666),
 ('doga', 0.3239491283893585),
 ('zanaat', 0.31868666410446167),
 ('sonrasını', 0.3117790222167969),
 ('guneselifiniz', 0.3080103397369385),
 ('weirdo', 0.30511805415153503),
 ('evlilikleri', 0.3033685088157654)]

# Decision Tree

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [102]:
X = df.iloc[:, 1:]
Y = df.loc[:, ["Tweets"]]

In [135]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [136]:
decision_tree = DecisionTreeClassifier()

In [137]:
decision_tree.fit(X_train, Y_train)

DecisionTreeClassifier()

In [138]:
Y_pred = decision_tree.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
f1_scores = f1_score(Y_test, Y_pred, average="weighted")

In [139]:
print(f"Accuracy Score: {accuracy}\nF1 Score: {f1_scores}")

Accuracy Score: 0.013289036544850499
F1 Score: 0.012181616832779622
